## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [10]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
#file_index='BBBSBBBB'
file_index='BSBSSSBS'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1651
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|            coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|-1143.5659350269993| -50.5755076903399|-181.52583650704696|     19.8|BSBSSSBS| 30.1064| -83.1819|       TOBS| 0.3394128117833675|0.33812073528054964|0.32147570382979035| 0.16

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [33]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
Features=', '.join(['station','coeff_1', 'coeff_2', 'coeff_3','elevation', 'latitude', 'longitude',\
          'res_1', 'res_2', 'res_3','res_mean', 'year'])
#Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, coeff_1, coeff_2, coeff_3, elevation, latitude, longitude, res_1, res_2, res_3, res_mean, year FROM weather


,station,coeff_1,coeff_2,coeff_3,elevation,latitude,longitude,res_1,res_2,res_3,res_mean,year
0,USC00085539,-1143.565935,-50.575508,-181.525837,19.8,30.1064,-83.1819,0.339413,0.338121,0.321476,0.165716,1995.0
1,USC00085539,-1293.151064,-74.741333,11.386451,19.8,30.1064,-83.1819,0.323303,0.321042,0.320990,0.223430,1996.0
2,USW00092806,833.076285,-15.883804,122.110402,2.4,27.7628,-82.6261,0.335383,0.335142,0.320862,0.040047,1948.0
3,USC00084723,-1176.034744,18.629096,90.235394,41.1,29.9453,-82.3314,0.324012,0.323842,0.319862,0.175524,2005.0
4,USC00083956,-1188.442254,-64.306919,210.792719,19.8,29.8286,-82.5972,0.341224,0.339591,0.319641,0.170750,2007.0


In [34]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station,USC00080535,USC00080598,USC00080940,USC00080945,USC00081046,USC00081163,USC00081310,USC00081432,USC00081632,USC00081635,...,USC00088530,USC00088565,USC00088573,USC00088782,USC00088824,USC00088890,USC00089120,USC00089176,USC00089430,USW00092806
year,,,,,,,,,,,,,,,,,,,,,
1901.0,NaN,NaN,NaN,NaN,-885.490163,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1902.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-233.104682,NaN,NaN,NaN,NaN,NaN
1903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-1013.729025,NaN,-169.780516,NaN,NaN,NaN,NaN,NaN
1904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-677.022329,NaN,-155.704196,NaN,NaN,NaN,NaN,NaN
1905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-890.142071,NaN,-102.603870,NaN,NaN,NaN,NaN,NaN
1906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-962.881657,NaN,-102.988970,NaN,NaN,NaN,NaN,NaN
1907.0,NaN,NaN,NaN,NaN,-240.258448,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,77.154130,NaN,NaN,NaN,NaN,NaN
1908.0,NaN,NaN,NaN,NaN,-118.728576,NaN,NaN,-169.247180,NaN,NaN,...,NaN,NaN,NaN,NaN,-6.393996,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,19.597811,NaN,NaN,-270.465009,NaN,NaN,...,NaN,NaN,NaN,NaN,-44.415968,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [35]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  610.88931291
RMS removing mean-by-station=  404.50008753
RMS removing mean-by-year   =  572.824138936


In [36]:
def MS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year)

total MS                   =  373185.752627
MS removing mean-by-station=  163620.320812
MS removing mean-by-year   =  328127.494148


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 195.727025297
0 after removing mean by year    = 131.31596864
0 after removing mean by stations= 116.264223899
1 after removing mean by year    = 115.679654512
1 after removing mean by stations= 115.539390957
2 after removing mean by year    = 115.48765606
2 after removing mean by stations= 115.465982349
3 after removing mean by year    = 115.456414624
3 after removing mean by stations= 115.45208948
4 after removing mean by year    = 115.450111848
4 after removing mean by stations= 115.449202429
